In [1]:
import tensorflow as tf
# Tensorflow即时执行模式（TF2默认使用，TF1中需调用tf.enable_eager_execution()函数才能启用）

In [3]:
random_float = tf.random.uniform(shape=())
random_float2 = tf.random.uniform(shape=([1]))
print(random_float)
print(random_float2)

tf.Tensor(0.90205395, shape=(), dtype=float32)
tf.Tensor([0.93093073], shape=(1,), dtype=float32)


In [4]:
A = tf.constant([[1, 2], [3, 4]])
print(A)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [5]:
print(A.shape)
print(A.dtype)
print(A.numpy())
# 查看张量的形状，类型和值

(2, 2)
<dtype: 'int32'>
[[1 2]
 [3 4]]


In [6]:
B = tf.constant([[5, 6], [7, 8]])

In [7]:
C = tf.add(A, B)        #矩阵加法
D = tf.matmul(A, B)     #矩阵内积
print(C)
print(D)

tf.Tensor(
[[ 6  8]
 [10 12]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)


In [8]:
# 计算函数导数
x = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
    y = tf.square(x)
y_grad = tape.gradient(y, x)
print(y, y_grad)

tf.Tensor(9.0, shape=(), dtype=float32) tf.Tensor(6.0, shape=(), dtype=float32)


In [9]:
# 多元函数求导
# 函数 L(w,b)=||Xw+b-y||^2 w=(1,2)^T, b=1
X = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.Variable(initial_value=[[1.], [2.]])
b = tf.Variable(initial_value=1.)

In [10]:
with tf.GradientTape() as tape:
    L = tf.reduce_sum(tf.square(tf.matmul(X, w)+b-y))
w_grad, b_grad = tape.gradient(L, [w, b])
print(L, w_grad)
print(L, b_grad)

tf.Tensor(125.0, shape=(), dtype=float32) tf.Tensor(
[[ 70.]
 [100.]], shape=(2, 1), dtype=float32)
tf.Tensor(125.0, shape=(), dtype=float32) tf.Tensor(30.0, shape=(), dtype=float32)


In [11]:
# TF简单实现线性回归
import numpy as np
X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
Y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

X = (X_raw-X_raw.min()) / (X_raw.max()-X_raw.min())
Y = (Y_raw-Y_raw.min()) / (Y_raw.max()-Y_raw.min())
print(X)
print(Y)

[0.   0.25 0.5  0.75 1.  ]
[0.         0.36363637 0.54545456 0.8181818  1.        ]


In [13]:
X = tf.constant(X)
Y = tf.constant(Y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

num_epoch = 1000
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
for epoch in range(num_epoch):
    # 使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = tf.reduce_sum(tf.square(y_pred - y))
    # TF自动计算损失函数关于自变量（即模型参数）的梯度
    grads = tape.gradient(loss, variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))

print(a, b)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.08963588> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.4503194>


In [3]:
import tensorflow as tf

In [4]:
path = tf.keras.utils.get_file("nietzsche.txt", origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
print(path)

606208/600901 [==============================] - 1s 2us/step
/home/opt/.keras/datasets/nietzsche.txt


In [7]:
! pip3 install zh

  Created wheel for zh: filename=zh-1.0.0-py3-none-any.whl size=1267 sha256=77c5a4f265d35f85f52c62f940aee299b52004e760852e301b1e5e2fce4462ae
  Stored in directory: /home/opt/.cache/pip/wheels/6e/a4/9d/15dfd9db4199cbb84abc69dc8d3c0648a9664ae3fa30f7218a
Successfully built zh


In [7]:
# 一个简单的训练，并保存加载模型流程
import tensorflow as tf
import numpy as np

In [10]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # flatten层会将除batch_size(第一维)外的维度合并
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
    
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        x = self.dense2(x)
        output = tf.nn.softmax(x)
        return output

In [5]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

In [6]:
def train(lr, batch_size, epoches, model_path, data_loader):
    model = MLP()
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    num_batches = int(data_loader.num_train_data // batch_size * epoches)
    checkpoint = tf.train.Checkpoint(myAwesomeModel=model)
    
    for batch_index in range(1, num_batches+1):
        X, y = data_loader.get_batch(batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(X)
            # TF2 Keras中有两个与交叉熵相关的损失函数：
            # tf.keras.losses.categorical_crossentropy和tf.keras.losses.sparse_categorical_crossentropy
            # 后者可以直接传入int类型的标签类别，它会将标签one_hot化
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
            loss = tf.reduce_mean(loss)
            print("batch: {}, loss: {}".format(batch_index, loss.numpy()))
        grads = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        if batch_index % 100 == 0:
            path = checkpoint.save(model_path)
            print("model saved to {}".format(path))

In [3]:
def test(model_root, data_loader):
    model = MLP()
    # 从保存的参数中恢复模型
    checkpoint = tf.train.Checkpoint(myAwesomeModel=model)
    checkpoint.restore(tf.train.latest_checkpoint(model_root))
    y_pred = np.argmax(model.call(data_loader.test_data), axis=-1)
    test_acc = sum(y_pred == data_loader.test_label) / data_loader.num_test_data
    print("test accuracy: {}".format(test_acc))

In [8]:
num_epoches = 20
batch_size = 64
learning_rate = 0.0001
data_loader = MNISTLoader()
model_path = "./saving/model.ckpt"
model_root = "./saving"

In [12]:
# 开始训练模型
train(learning_rate, batch_size, num_epoches, model_path, data_loader)

batch: 1, loss: 2.325194835662842
batch: 2, loss: 2.389965295791626
batch: 3, loss: 2.4046218395233154
batch: 4, loss: 2.3886492252349854
batch: 5, loss: 2.5222110748291016
batch: 6, loss: 2.311737060546875
batch: 7, loss: 2.3218295574188232
batch: 8, loss: 2.3464279174804688
batch: 9, loss: 2.297900676727295
batch: 10, loss: 2.3356008529663086
batch: 11, loss: 2.3765928745269775
batch: 12, loss: 2.2525863647460938
batch: 13, loss: 2.404512643814087
batch: 14, loss: 2.28544282913208
batch: 15, loss: 2.334932327270508
batch: 16, loss: 2.200711488723755
batch: 17, loss: 2.2805228233337402
batch: 18, loss: 2.238523483276367
batch: 19, loss: 2.254483699798584
batch: 20, loss: 2.2005114555358887
batch: 21, loss: 2.179486036300659
batch: 22, loss: 2.198049545288086
batch: 23, loss: 2.1824395656585693
batch: 24, loss: 2.08872389793396
batch: 25, loss: 2.25022029876709
batch: 26, loss: 2.1311352252960205
batch: 27, loss: 2.1372039318084717
batch: 28, loss: 2.181314468383789
batch: 29, loss: 2.

batch: 232, loss: 0.9153490662574768
batch: 233, loss: 0.9473258852958679
batch: 234, loss: 0.8783504366874695
batch: 235, loss: 1.0314772129058838
batch: 236, loss: 1.0794472694396973
batch: 237, loss: 0.9361597895622253
batch: 238, loss: 0.902553141117096
batch: 239, loss: 1.0961575508117676
batch: 240, loss: 0.9489743113517761
batch: 241, loss: 0.824978232383728
batch: 242, loss: 0.8985188007354736
batch: 243, loss: 0.9972012042999268
batch: 244, loss: 0.8999552726745605
batch: 245, loss: 0.9459249973297119
batch: 246, loss: 0.8964728713035583
batch: 247, loss: 0.9977432489395142
batch: 248, loss: 0.9202373027801514
batch: 249, loss: 0.8639922142028809
batch: 250, loss: 0.8593462109565735
batch: 251, loss: 0.8082056045532227
batch: 252, loss: 0.8696210384368896
batch: 253, loss: 0.8177279233932495
batch: 254, loss: 0.7340137958526611
batch: 255, loss: 0.9901547431945801
batch: 256, loss: 0.8509083986282349
batch: 257, loss: 0.9095944166183472
batch: 258, loss: 0.7867751121520996
bat

batch: 471, loss: 0.6976512670516968
batch: 472, loss: 0.6713790893554688
batch: 473, loss: 0.4526668190956116
batch: 474, loss: 0.4625406265258789
batch: 475, loss: 0.5723934173583984
batch: 476, loss: 0.7038400173187256
batch: 477, loss: 0.7028857469558716
batch: 478, loss: 0.5926361083984375
batch: 479, loss: 0.5913316011428833
batch: 480, loss: 0.8051643967628479
batch: 481, loss: 0.6355253458023071
batch: 482, loss: 0.6942131519317627
batch: 483, loss: 0.5657846331596375
batch: 484, loss: 0.4419146776199341
batch: 485, loss: 0.7403565645217896
batch: 486, loss: 0.6423547863960266
batch: 487, loss: 0.5630885362625122
batch: 488, loss: 0.6361466646194458
batch: 489, loss: 0.5762280821800232
batch: 490, loss: 0.559424102306366
batch: 491, loss: 0.5636579990386963
batch: 492, loss: 0.5869840383529663
batch: 493, loss: 0.4984332025051117
batch: 494, loss: 0.6022838354110718
batch: 495, loss: 0.523472011089325
batch: 496, loss: 0.5811885595321655
batch: 497, loss: 0.632291853427887
batc

batch: 704, loss: 0.3927021324634552
batch: 705, loss: 0.5872839689254761
batch: 706, loss: 0.5366857647895813
batch: 707, loss: 0.5108003616333008
batch: 708, loss: 0.52535480260849
batch: 709, loss: 0.535590410232544
batch: 710, loss: 0.6593323945999146
batch: 711, loss: 0.524250864982605
batch: 712, loss: 0.5286321640014648
batch: 713, loss: 0.5350164175033569
batch: 714, loss: 0.4682174026966095
batch: 715, loss: 0.3500710427761078
batch: 716, loss: 0.4571795165538788
batch: 717, loss: 0.439942330121994
batch: 718, loss: 0.5061509609222412
batch: 719, loss: 0.3944440484046936
batch: 720, loss: 0.5777555704116821
batch: 721, loss: 0.41974514722824097
batch: 722, loss: 0.45940738916397095
batch: 723, loss: 0.4874120354652405
batch: 724, loss: 0.4545799791812897
batch: 725, loss: 0.4595556855201721
batch: 726, loss: 0.5065666437149048
batch: 727, loss: 0.5035240650177002
batch: 728, loss: 0.3831554055213928
batch: 729, loss: 0.43563687801361084
batch: 730, loss: 0.43350157141685486
ba

batch: 936, loss: 0.35215258598327637
batch: 937, loss: 0.5231376886367798
batch: 938, loss: 0.5618042349815369
batch: 939, loss: 0.42729678750038147
batch: 940, loss: 0.5354726314544678
batch: 941, loss: 0.4637874364852905
batch: 942, loss: 0.2887837290763855
batch: 943, loss: 0.4660528302192688
batch: 944, loss: 0.48122337460517883
batch: 945, loss: 0.3712397515773773
batch: 946, loss: 0.37483295798301697
batch: 947, loss: 0.4457261264324188
batch: 948, loss: 0.5132746696472168
batch: 949, loss: 0.42343100905418396
batch: 950, loss: 0.5399971008300781
batch: 951, loss: 0.2473340928554535
batch: 952, loss: 0.45702478289604187
batch: 953, loss: 0.5188466310501099
batch: 954, loss: 0.30947211384773254
batch: 955, loss: 0.37731051445007324
batch: 956, loss: 0.3257981538772583
batch: 957, loss: 0.40166354179382324
batch: 958, loss: 0.38127392530441284
batch: 959, loss: 0.24734972417354584
batch: 960, loss: 0.5011435747146606
batch: 961, loss: 0.48490577936172485
batch: 962, loss: 0.305016

...

batch: 16123, loss: 0.12801720201969147
batch: 16124, loss: 0.1560162901878357
batch: 16125, loss: 0.07510773837566376
batch: 16126, loss: 0.14186586439609528
batch: 16127, loss: 0.1003318578004837
batch: 16128, loss: 0.05624959245324135
batch: 16129, loss: 0.04077252000570297
batch: 16130, loss: 0.23056456446647644
batch: 16131, loss: 0.11861799657344818
batch: 16132, loss: 0.060545969754457474
batch: 16133, loss: 0.07347141206264496
batch: 16134, loss: 0.036544088274240494
batch: 16135, loss: 0.24504998326301575
batch: 16136, loss: 0.12387961149215698
batch: 16137, loss: 0.04759715124964714
batch: 16138, loss: 0.20787501335144043
batch: 16139, loss: 0.13777516782283783
batch: 16140, loss: 0.0974886417388916
batch: 16141, loss: 0.09694423526525497
batch: 16142, loss: 0.037544362246990204
batch: 16143, loss: 0.0923214852809906
batch: 16144, loss: 0.08103849738836288
batch: 16145, loss: 0.14598965644836426
batch: 16146, loss: 0.12129972875118256
batch: 16147, loss: 0.049572285264730453


batch: 16331, loss: 0.0770532563328743
batch: 16332, loss: 0.11271318793296814
batch: 16333, loss: 0.10283634811639786
batch: 16334, loss: 0.13374142348766327
batch: 16335, loss: 0.08194156736135483
batch: 16336, loss: 0.10021995007991791
batch: 16337, loss: 0.20531317591667175
batch: 16338, loss: 0.05963430926203728
batch: 16339, loss: 0.02356889843940735
batch: 16340, loss: 0.08773032575845718
batch: 16341, loss: 0.144869863986969
batch: 16342, loss: 0.027783378958702087
batch: 16343, loss: 0.1268925815820694
batch: 16344, loss: 0.09995894879102707
batch: 16345, loss: 0.12113026529550552
batch: 16346, loss: 0.11883515864610672
batch: 16347, loss: 0.03764001280069351
batch: 16348, loss: 0.12634915113449097
batch: 16349, loss: 0.07079005986452103
batch: 16350, loss: 0.04971779137849808
batch: 16351, loss: 0.028478559106588364
batch: 16352, loss: 0.05408691614866257
batch: 16353, loss: 0.11175014078617096
batch: 16354, loss: 0.29663345217704773
batch: 16355, loss: 0.20993170142173767
ba

batch: 16543, loss: 0.048430535942316055
batch: 16544, loss: 0.06573960930109024
batch: 16545, loss: 0.036980438977479935
batch: 16546, loss: 0.06068973243236542
batch: 16547, loss: 0.11831261217594147
batch: 16548, loss: 0.1342845857143402
batch: 16549, loss: 0.041691698133945465
batch: 16550, loss: 0.16485485434532166
batch: 16551, loss: 0.04883544147014618
batch: 16552, loss: 0.08924716711044312
batch: 16553, loss: 0.04234666749835014
batch: 16554, loss: 0.0523047149181366
batch: 16555, loss: 0.1944560408592224
batch: 16556, loss: 0.16423140466213226
batch: 16557, loss: 0.21713782846927643
batch: 16558, loss: 0.16986870765686035
batch: 16559, loss: 0.031160490587353706
batch: 16560, loss: 0.10991169512271881
batch: 16561, loss: 0.1212160587310791
batch: 16562, loss: 0.10292176902294159
batch: 16563, loss: 0.09986409544944763
batch: 16564, loss: 0.0897592231631279
batch: 16565, loss: 0.13058722019195557
batch: 16566, loss: 0.09883561730384827
batch: 16567, loss: 0.2613949775695801
ba

batch: 16759, loss: 0.061570197343826294
batch: 16760, loss: 0.12281922250986099
batch: 16761, loss: 0.0657401755452156
batch: 16762, loss: 0.08856354653835297
batch: 16763, loss: 0.10530219972133636
batch: 16764, loss: 0.11981216818094254
batch: 16765, loss: 0.11481578648090363
batch: 16766, loss: 0.1364961862564087
batch: 16767, loss: 0.16212385892868042
batch: 16768, loss: 0.03272957354784012
batch: 16769, loss: 0.09914644062519073
batch: 16770, loss: 0.043876856565475464
batch: 16771, loss: 0.12192586809396744
batch: 16772, loss: 0.1432906538248062
batch: 16773, loss: 0.14183828234672546
batch: 16774, loss: 0.08338013291358948
batch: 16775, loss: 0.041800606995821
batch: 16776, loss: 0.06965514272451401
batch: 16777, loss: 0.07916810363531113
batch: 16778, loss: 0.038136862218379974
batch: 16779, loss: 0.058261506259441376
batch: 16780, loss: 0.05610690638422966
batch: 16781, loss: 0.1136297658085823
batch: 16782, loss: 0.35295096039772034
batch: 16783, loss: 0.15742653608322144
ba

KeyboardInterrupt: 

In [11]:
# 测试模型结果
test(model_root, data_loader)

test accuracy: 0.9665


In [1]:
import tensorflow as tf
# GPU设置显存使用策略
# TF2提供两种显存使用策略，让我们能够更灵活地控制程序地显存使用方式：
# (1)仅在需要时申请显存空间(程序初始运行时消耗很少地显存，随着程序的运行而动态申请显存)
gpus = tf.config.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(device=gpu, enable=True)

# (2)限制消耗固定大小的显存(程序不会超出限定的显存大小，若超出则报错)
tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=y1024)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [12]:
import tensorflow as tf

num_epochs = 1
batch_size = 50
learning_rate = 0.001

In [13]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

In [14]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
])
data_loader = MNISTLoader()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)
model.fit(data_loader.train_data, data_loader.train_label, epochs=num_epochs, batch_size=batch_size)
tf.saved_model.save(model, "saved/1")

1200/1200 [==============================] - 1s 1ms/step - loss: 0.2943 - sparse_categorical_accuracy: 0.9179
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved/1/assets


In [15]:
batch_size = 50

model = tf.saved_model.load("saved/1")
data_loader = MNISTLoader()
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.949700


In [16]:
data_loader.test_data[0:3]

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [17]:
print(data_loader.test_data[0:3].shape)

(3, 28, 28, 1)
